In [1]:
import pandas as pd
import bz2
import xml.etree.ElementTree as ET
import re
import os
import glob
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

pd.set_option('display.max_colwidth', None)

In [2]:
BASE_DIR = Path("")  
DOT_DIR = BASE_DIR / "dot_files"  
JSON_DIR = BASE_DIR / "json-files"

## 1. Data Loading and Analysis Functions


In [3]:
def analyze_xcfa_file(filepath):
    """Analyze a single XCFA JSON file and extract metrics."""
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
    except (json.JSONDecodeError, FileNotFoundError) as e:
        print(f"Error loading {filepath}: {e}")
        return None
    
    analysis = {
        'filename': Path(filepath).name,
        'total_variables': 0,
        'total_locations': 0, 
        'total_edges': 0,
        'main_procedure_edges': 0,
        'sequence_label_edges': 0,
        'total_sequence_labels': 0,
        'procedures_count': len(data.get('procedures', [])),
        'file_size_kb': Path(filepath).stat().st_size / 1024
    }
    
    # Analyze procedures
    for procedure in data.get('procedures', []):
        proc_name = procedure.get('name', 'unnamed')
        vars_count = len(procedure.get('vars', []))
        locs_count = len(procedure.get('locs', []))
        edges_count = len(procedure.get('edges', []))
        
        analysis['total_variables'] += vars_count
        analysis['total_locations'] += locs_count
        analysis['total_edges'] += edges_count
        
        # Count sequence labels in main procedure
        if proc_name == 'main':
            analysis['main_procedure_edges'] = edges_count
            for edge in procedure.get('edges', []):
                label = edge.get('label', {})
                if label.get('type') == 'hu.bme.mit.theta.xcfa.model.SequenceLabel':
                    analysis['sequence_label_edges'] += 1
                    analysis['total_sequence_labels'] += len(label.get('labels', []))
    
    # Calculate averages
    if analysis['sequence_label_edges'] > 0:
        analysis['avg_labels_per_sequence_edge'] = (
            analysis['total_sequence_labels'] / analysis['sequence_label_edges']
        )
    else:
        analysis['avg_labels_per_sequence_edge'] = 0
    
    return analysis

def analyze_file_pairs(btor2_dir, c_dir, max_files=None):
    """Analyze pairs of BTOR2 and C XCFA files."""
    btor2_files = list(Path(btor2_dir).glob('*.json'))
    c_files = list(Path(c_dir).glob('*.json'))
    
    if max_files:
        btor2_files = btor2_files[:max_files]
        c_files = c_files[:max_files]
    
    results = []
    
    for btor2_file, c_file in zip(btor2_files, c_files):
        btor2_analysis = analyze_xcfa_file(btor2_file)
        c_analysis = analyze_xcfa_file(c_file)
        
        if btor2_analysis and c_analysis:
            # Calculate ratios and combined metrics
            pair_result = {
                'file_id': btor2_analysis['filename'].replace('.json', ''),
                'btor2_variables': btor2_analysis['total_variables'],
                'c_variables': c_analysis['total_variables'],
                'variables_ratio': safe_divide(c_analysis['total_variables'], btor2_analysis['total_variables']),
                
                'btor2_locations': btor2_analysis['total_locations'],
                'c_locations': c_analysis['total_locations'], 
                'locations_ratio': safe_divide(c_analysis['total_locations'], btor2_analysis['total_locations']),
                
                'btor2_edges': btor2_analysis['total_edges'],
                'c_edges': c_analysis['total_edges'],
                'edges_ratio': safe_divide(c_analysis['total_edges'], btor2_analysis['total_edges']),
                
                'btor2_sequence_edges': btor2_analysis['sequence_label_edges'],
                'c_sequence_edges': c_analysis['sequence_label_edges'],
                'sequence_edges_ratio': safe_divide(c_analysis['sequence_label_edges'], btor2_analysis['sequence_label_edges']),
                
                'btor2_avg_labels': btor2_analysis['avg_labels_per_sequence_edge'],
                'c_avg_labels': c_analysis['avg_labels_per_sequence_edge'],
                'avg_labels_ratio': safe_divide(c_analysis['avg_labels_per_sequence_edge'], btor2_analysis['avg_labels_per_sequence_edge']),
                
                'btor2_procedures': btor2_analysis['procedures_count'],
                'c_procedures': c_analysis['procedures_count'],
                'procedures_ratio': safe_divide(c_analysis['procedures_count'], btor2_analysis['procedures_count']),
                
                'btor2_file_size': btor2_analysis['file_size_kb'],
                'c_file_size': c_analysis['file_size_kb'],
                'file_size_ratio': safe_divide(c_analysis['file_size_kb'], btor2_analysis['file_size_kb'])
            }
            results.append(pair_result)
    
    return pd.DataFrame(results)

def safe_divide(a, b):
    """Safe division to handle zero values."""
    return a / b if b != 0 else float('inf')

# Example usage with sample data
print("Loading sample data...")
# In practice, you would point to your actual directories
# df = analyze_file_pairs('path/to/btor2/files', 'path/to/c/files', max_files=100)

Loading sample data...


## 2. Load the Data

In [4]:
# TODO

sample_data = []
for i in range(n_files):
    sample_data.append()

df = pd.DataFrame(sample_data)
print(f"Generated sample data for {len(df)} file pairs")
print(df.head())

ZeroDivisionError: division by zero

## 3. Statistical Summary


In [ ]:
# Basic statistics
print("STATISTICAL SUMMARY")
print("=" * 50)

metrics = ['variables', 'locations', 'edges', 'sequence_edges', 'procedures', 'file_size']
summary_data = []

for metric in metrics:
    btor2_mean = df[f'btor2_{metric}'].mean()
    c_mean = df[f'c_{metric}'].mean()
    ratio_mean = df[f'{metric}_ratio'].mean()
    
    summary_data.append({
        'Metric': metric.title(),
        'BTOR2_Mean': btor2_mean,
        'C_Mean': c_mean,
        'C/BTOR2_Ratio': ratio_mean,
        'BTOR2_Median': df[f'btor2_{metric}'].median(),
        'C_Median': df[f'c_{metric}'].median(),
        'Ratio_Median': df[f'{metric}_ratio'].median()
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.round(2))

## 4. Distribution Comparison Plots


In [ ]:
# Create distribution comparison plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

metrics = ['variables', 'locations', 'edges', 'sequence_edges', 'procedures', 'file_size']
titles = ['Variables', 'Locations', 'Edges', 'Sequence Edges', 'Procedures', 'File Size (KB)']

for i, (metric, title) in enumerate(zip(metrics, titles)):
    # Box plot
    data = pd.DataFrame({
        'Value': list(df[f'btor2_{metric}']) + list(df[f'c_{metric}']),
        'Type': ['BTOR2'] * len(df) + ['C'] * len(df)
    })
    
    sns.boxplot(x='Type', y='Value', data=data, ax=axes[i])
    axes[i].set_title(f'Distribution of {title}')
    axes[i].set_ylabel(title)
    axes[i].set_yscale('log')  # Log scale for better visualization

plt.suptitle('Distribution Comparison: BTOR2 vs C XCFA Files', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

## 5. Ratio Analysis


In [ ]:
# Ratio analysis
ratio_metrics = ['variables_ratio', 'locations_ratio', 'edges_ratio', 
                 'sequence_edges_ratio', 'procedures_ratio', 'file_size_ratio']
ratio_titles = ['Variables Ratio', 'Locations Ratio', 'Edges Ratio', 
                'Sequence Edges Ratio', 'Procedures Ratio', 'File Size Ratio']

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, (metric, title) in enumerate(zip(ratio_metrics, ratio_titles)):
    # Remove infinite values for plotting
    ratios = df[metric].replace([np.inf, -np.inf], np.nan).dropna()
    
    # Histogram of ratios
    axes[i].hist(ratios, bins=30, alpha=0.7, edgecolor='black')
    axes[i].axvline(x=ratios.median(), color='red', linestyle='--', 
                   label=f'Median: {ratios.median():.2f}x')
    axes[i].axvline(x=1, color='green', linestyle='-', alpha=0.5, label='1x (Equal)')
    axes[i].set_xlabel(f'C / BTOR2 Ratio')
    axes[i].set_ylabel('Frequency')
    axes[i].set_title(f'Distribution of {title}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Complexity Ratio Analysis: C vs BTOR2 XCFA Files', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

## 6. Cumulative Distribution Functions (CDFs)


In [ ]:
# CDF plots for detailed distribution analysis
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

metrics = ['variables', 'locations', 'edges', 'sequence_edges', 'procedures', 'file_size']
titles = ['Variables', 'Locations', 'Edges', 'Sequence Edges', 'Procedures', 'File Size (KB)']

for i, (metric, title) in enumerate(zip(metrics, titles)):
    btor2_vals = sorted(df[f'btor2_{metric}'])
    c_vals = sorted(df[f'c_{metric}'])
    
    # CDF plot
    axes[i].plot(btor2_vals, np.linspace(0, 1, len(btor2_vals)), 
                label='BTOR2', linewidth=2)
    axes[i].plot(c_vals, np.linspace(0, 1, len(c_vals)), 
                label='C', linewidth=2)
    axes[i].set_xlabel(title)
    axes[i].set_ylabel('CDF')
    axes[i].set_title(f'CDF of {title}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)
    axes[i].set_xscale('log')

plt.suptitle('Cumulative Distribution Functions: BTOR2 vs C XCFA Files', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

## 7. Scatter Plot Matrix


In [ ]:
# Scatter plot matrix to show correlations
metrics = ['btor2_variables', 'btor2_locations', 'btor2_edges', 'c_variables', 'c_locations', 'c_edges']
scatter_df = df[metrics].copy()
scatter_df.columns = [col.replace('btor2_', 'B_').replace('c_', 'C_') for col in scatter_df.columns]

# Create scatter matrix
fig = pd.plotting.scatter_matrix(scatter_df, figsize=(16, 16), diagonal='hist', alpha=0.7)
plt.suptitle('Scatter Matrix: BTOR2 vs C XCFA Metrics', fontsize=20, y=0.92)
plt.tight_layout()
plt.show()

## 8. Complexity Ratio Heatmap


In [ ]:
# Create complexity ratio heatmap by size categories
def create_complexity_heatmap(df):
    """Create heatmap showing complexity ratios across size categories."""
    
    # Define size categories based on BTOR2 variables
    df['size_category'] = pd.cut(df['btor2_variables'], 
                                bins=[0, 10, 50, 100, np.inf],
                                labels=['Small\n(0-10 vars)', 'Medium\n(10-50 vars)', 
                                       'Large\n(50-100 vars)', 'Very Large\n(100+ vars)'])
    
    ratio_metrics = ['variables_ratio', 'locations_ratio', 'edges_ratio', 'file_size_ratio']
    metric_names = ['Variables', 'Locations', 'Edges', 'File Size']
    
    # Calculate mean ratios for each category
    heatmap_data = []
    categories = df['size_category'].cat.categories
    
    for category in categories:
        category_data = df[df['size_category'] == category]
        ratios = []
        for metric in ratio_metrics:
            # Remove infinite values and calculate mean
            valid_ratios = category_data[metric].replace([np.inf, -np.inf], np.nan).dropna()
            ratios.append(valid_ratios.mean() if len(valid_ratios) > 0 else np.nan)
        heatmap_data.append(ratios)
    
    # Create heatmap
    fig, ax = plt.subplots(figsize=(12, 8))
    im = ax.imshow(heatmap_data, cmap='RdYlBu_r', aspect='auto', vmin=1, vmax=5)
    
    # Add labels and annotations
    ax.set_xticks(range(len(metric_names)))
    ax.set_yticks(range(len(categories)))
    ax.set_xticklabels(metric_names)
    ax.set_yticklabels(categories)
    
    # Add text annotations
    for i in range(len(categories)):
        for j in range(len(metric_names)):
            if not np.isnan(heatmap_data[i][j]):
                text = ax.text(j, i, f'{heatmap_data[i][j]:.1f}x',
                             ha="center", va="center", color="black", 
                             fontweight='bold', fontsize=12)
    
    ax.set_title('C-to-BTOR2 Complexity Ratios by Size Category', fontsize=16, pad=20)
    plt.colorbar(im, ax=ax, label='Complexity Ratio (C / BTOR2)')
    plt.tight_layout()
    plt.show()

create_complexity_heatmap(df)

## 9. Statistical Dashboard


In [ ]:
# Create comprehensive statistical dashboard
def create_statistical_dashboard(df):
    """Create a comprehensive statistical summary dashboard."""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Mean comparison bar plot
    metrics = ['variables', 'locations', 'edges']
    btor2_means = [df[f'btor2_{metric}'].mean() for metric in metrics]
    c_means = [df[f'c_{metric}'].mean() for metric in metrics]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    ax1.bar(x - width/2, btor2_means, width, label='BTOR2', alpha=0.7, color='skyblue')
    ax1.bar(x + width/2, c_means, width, label='C', alpha=0.7, color='lightcoral')
    ax1.set_ylabel('Mean Value')
    ax1.set_title('Mean Complexity Comparison')
    ax1.set_xticks(x)
    ax1.set_xticklabels(['Variables', 'Locations', 'Edges'])
    ax1.legend()
    ax1.set_yscale('log')
    ax1.grid(True, alpha=0.3)
    
    # 2. Ratio distribution
    ratio_data = []
    for metric in metrics:
        ratios = df[f'{metric}_ratio'].replace([np.inf, -np.inf], np.nan).dropna()
        ratio_data.append(ratios)
    
    ax2.boxplot(ratio_data, labels=['Variables', 'Locations', 'Edges'])
    ax2.axhline(y=1, color='red', linestyle='--', alpha=0.7, label='Equal Complexity')
    ax2.set_ylabel('C / BTOR2 Ratio')
    ax2.set_title('Complexity Ratio Distribution')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Correlation heatmap
    correlation_metrics = ['btor2_variables', 'btor2_locations', 'btor2_edges',
                         'c_variables', 'c_locations', 'c_edges',
                         'variables_ratio', 'locations_ratio', 'edges_ratio']
    
    corr_matrix = df[correlation_metrics].corr()
    # Mask upper triangle
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    
    sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
                center=0, ax=ax3, square=True, cbar_kws={'shrink': 0.8})
    ax3.set_title('Correlation Matrix of XCFA Metrics')
    
    # 4. Outlier analysis
    outlier_counts = {}
    for metric in ['variables_ratio', 'locations_ratio', 'edges_ratio']:
        ratios = df[metric].replace([np.inf, -np.inf], np.nan).dropna()
        Q1 = ratios.quantile(0.25)
        Q3 = ratios.quantile(0.75)
        IQR = Q3 - Q1
        outliers = ratios[(ratios < (Q1 - 1.5 * IQR)) | (ratios > (Q3 + 1.5 * IQR))]
        outlier_counts[metric.replace('_ratio', '')] = len(outliers)
    
    ax4.bar(outlier_counts.keys(), outlier_counts.values(), color='orange', alpha=0.7)
    ax4.set_ylabel('Number of Outliers')
    ax4.set_title('Outlier Count by Metric (IQR Method)')
    ax4.grid(True, alpha=0.3)
    
    plt.suptitle('XCFA Analysis: Comprehensive Statistical Dashboard', fontsize=18, y=0.95)
    plt.tight_layout()
    plt.show()

create_statistical_dashboard(df)

## 10. Key Insights and Conclusions


In [ ]:
# Generate key insights
print("KEY INSIGHTS AND CONCLUSIONS")
print("=" * 50)

# Calculate overall statistics
total_files = len(df)
high_complexity_files = len(df[df['variables_ratio'] > 3])
avg_ratio_vars = df['variables_ratio'].replace([np.inf, -np.inf], np.nan).mean()
avg_ratio_locs = df['locations_ratio'].replace([np.inf, -np.inf], np.nan).mean()
avg_ratio_edges = df['edges_ratio'].replace([np.inf, -np.inf], np.nan).mean()

print(f"📊 Dataset Overview:")
print(f"   • Total file pairs analyzed: {total_files}")
print(f"   • Files with >3x C complexity: {high_complexity_files} ({high_complexity_files/total_files*100:.1f}%)")

print(f"\n📈 Average Complexity Ratios (C / BTOR2):")
print(f"   • Variables: {avg_ratio_vars:.2f}x")
print(f"   • Locations: {avg_ratio_locs:.2f}x") 
print(f"   • Edges: {avg_ratio_edges:.2f}x")

print(f"\n🎯 Key Findings:")
print(f"   • C-generated XCFAs are typically {avg_ratio_vars:.1f}-{avg_ratio_edges:.1f}x larger than BTOR2 equivalents")
print(f"   • This overhead comes from intermediate variables and procedure call handling")
print(f"   • BTOR2 versions are more optimized for formal verification")

print(f"\n💡 Recommendations:")
print(f"   • Use BTOR2-generated XCFAs for formal verification tasks")
print(f"   • Consider C versions only when source-level debugging is needed")
print(f"   • The complexity ratio can help estimate verification time differences")

## 11. Export Results


In [ ]:
# Export analysis results
def export_analysis_results(df, summary_df):
    """Export analysis results to files."""
    
    # Save detailed data
    df.to_csv('xcfa_analysis_detailed.csv', index=False)
    summary_df.to_csv('xcfa_analysis_summary.csv', index=False)
    
    # Save key statistics
    with open('xcfa_analysis_insights.txt', 'w') as f:
        f.write("XCFA Analysis Insights\n")
        f.write("=" * 50 + "\n\n")
        
        f.write("Average Complexity Ratios (C / BTOR2):\n")
        for metric in ['variables', 'locations', 'edges']:
            ratio = df[f'{metric}_ratio'].replace([np.inf, -np.inf], np.nan).mean()
            f.write(f"  {metric.title()}: {ratio:.2f}x\n")
        
        f.write(f"\nTotal files analyzed: {len(df)}\n")
        
    print("✅ Analysis results exported to:")
    print("   - xcfa_analysis_detailed.csv")
    print("   - xcfa_analysis_summary.csv") 
    print("   - xcfa_analysis_insights.txt")

# Export results
export_analysis_results(df, summary_df)